Workflow for co-registering images with valis after running the pre-processing steps 
Pre-processing in comet.py and ultivue.py, submitted to cluster with submit_savetoVIPs.sh
"""
------------------------------------------------------------------------------
Perform image co-registration using the VALIS library after preprocessing with
`comet.py` and `ultivue.py`. Supports both rigid and high-resolution micro
non-rigid registration to align COMET, Ultivue, and H&E pseudo-DAPI images.

------------------------------------------------------------------------------
Inputs
------------------------------------------------------------------------------
- Preprocessed image folders (from comet.py / ultivue.py) containing:
------------------------------------------------------------------------------
Outputs
------------------------------------------------------------------------------
- Registration results written under each sample folder:
  • /registration/   (rigid registration results)
  • /micro_registration/ (high-resolution micro non-rigid registration results)
- Dataframes with registration error metrics (error_df, micro_error_df)
- Pickled registrar objects: <sample>_registrar.pickle
- Transformed aligned images accessible via VALIS API
------------------------------------------------------------------------------
"""



The section below is only for running on cluster, skip to next section for local running

In [ ]:
#ensuure pyvips can  find libvips installation (downloaded and compiled from source)
#export PATH="$HOME/local/libvips/bin:$PATH"
#export PKG_CONFIG_PATH="$HOME/local/libvips/lib64/pkgconfig:$PKG_CONFIG_PATH"
#export LD_LIBRARY_PATH="$HOME/local/libvips/lib64:/home/lythgo02/micromamba/pkgs/libffi-3.4.6-h2dba641_1/lib:/home/lythgo02/micromamba/envs/napari-env/lib$LD_LIBRARY_PATH"

#micromamba install python=3.10 scikit-image pandas pyvips -c conda-forge
#if you also install numpy via micromamba, make sure it is an older version compatible with valis numpy==1.23.5
#micromamba activate valis
#echo 

#pip install valis-wsi 
#pip install numpy==1.23.5 ipykernel
#micromamba install maven ???
#micromamba install openslide ???


#register kernel
#python -m ipykernel install --user --name valis --display-name "valis"
#select valis kernel

#tell which java to use (requires jdk bundle)
#export JAVA_HOME=/home/bioinformatics/software/java/jdk-17.0.3.1/bin/java 

#check
#echo $JAVA_HOME

The following section is for running locally

In [ ]:
!bash mamba activate valis-env
!python

Initialise parameter responsible for determining the rigid registration parameters, max_processed_image_dim_px by passing to the script during registration. Default = 850, adjust if registration fails (500-2000). When there is little empty space around tissue smaller values are better.Higher values for when there is more space to improve resolution. 

In [ ]:

# Import packages
import time
import os
import ctypes
import pandas as pd
import gc

from pathlib import Path
from valis import registration
import numpy as np

from valis.micro_rigid_registrar import MicroRigidRegistrar

ImportError: cannot import name 'registration' from 'valis' (unknown location)

In [ ]:

#input_dir='/home/lythgo02/Documents/valis_prep_ultivue0.9_hne20x_processed/'
input_dir='/home/lythgo02/Documents/comet_fullres/'
for file_id in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file_id)
    if not os.path.isdir(file_path):
        continue
    output_dir = os.path.join(input_dir,'registration')
    registrar = registration.Valis(
        file_path,
        output_dir,
        reference_img_f='pseudo-dapi-ultivue.tif',
        align_to_reference=True,
        image_type='flourescence'
    )
    registrar.register()
      #  Exits the loop after first iteration
  


In [ ]:

from valis.micro_rigid_registrar import MicroRigidRegistrar  # High-res rigid registration

micro_reg_fraction = 0.25  # Use 25% of full res for micro non-rigid

#input_dir='/home/lythgo02/Documents/valis_prep_cometlayer3_hne40x_ultivue0.9/'
#input_dir='/home/lythgo02/Documents/valis_prep_ultivue0.9_hne40x_cometlayer1/micro/'
input_dir='/home/lythgo02/Documents/comet_fullres/'
for file_id in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file_id)
    if not os.path.isdir(file_path):
        continue
    output_dir = os.path.join(input_dir,'micro_registration')
    registrar = registration.Valis(
        file_path,
        output_dir,
        image_type="flourescence",
        reference_img_f="pseudo-dapi-ultivue.tif",
        align_to_reference=True,
        micro_rigid_registrar_cls=MicroRigidRegistrar  # High-res rigid registration
    )
    rigid_registrar, non_rigid_registrar, error_df = registrar.register() # Step 1: Rigid registration (high-resolution)
    img_dims = np.array([slide_obj.slide_dimensions_wh[0] for slide_obj in registrar.slide_dict.values()]) # Step 2: Calculate size for micro non-rigid registration
    min_max_size = np.min([np.max(d) for d in img_dims])
    img_areas = [np.multiply(*d) for d in img_dims]
    max_img_w, max_img_h = tuple(img_dims[np.argmax(img_areas)])
    micro_reg_size = np.floor(min_max_size * micro_reg_fraction).astype(int)    
    micro_registrar, micro_error_df = registrar.register_micro(max_non_rigid_registration_dim_px=micro_reg_size)# Step 3: Micro non-rigid registration at higher resolution

#using micro-resolution registration (high resolution but more resource demanding)

# Set your paths




In [ ]:
#be sure to kill the java virtual machine at the end of the scripts
registration.kill_jvm()

see email from Thierry for the code he provided for the next steps

In [ ]:

file = 'FILENAME OF DATA'
wsi_hne = [] # Load numpy array containing the image on which to apply the spatial transform

# Load pre-computed registrar
registrar = registration.load_registrar(results_dst_dir+file+'/data/'+file+'_registrar.pickle')
reference_slide = registrar.get_slide('ihc_2')

for slide_obj in registrar.slide_dict.values():
    if slide_obj == reference_slide:
        continue

    # Apply co-registration to this different image of same size
    coreg_img = reference_slide.warp_img_from_to(wsi_hne,
                                    to_slide_obj=slide_obj,
                                    interp_method="nearest")
    
registration.kill_jvm()
